In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import sys
import os
if os.path.abspath('../') not in sys.path:
    sys.path.append(os.path.abspath('../'))
if os.path.abspath('../../tt_keras') not in sys.path:
    sys.path.append(os.path.abspath('../../tt_keras'))

if os.path.abspath('../../t3f') not in sys.path:
    sys.path.append(os.path.abspath('../../t3f'))

import automatic_speech_recognition as asr
import time
from datetime import datetime
import argparse
import pickle

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm.notebook import tqdm
from h5_to_tflite import TF_CUSTOM_OBJECTS

In [3]:
from tensorflow import keras
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.mixed_precision import experimental as mixed_precision

In [4]:
import horovod.tensorflow.keras as hvd

In [5]:
#%load_ext tensorboard
#%tensorboard --logdir=./models/ --port=32779

In [6]:
%pdb on

Automatic pdb calling has been turned ON


## Train

In [7]:
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [8]:
# Initialize Horovod
hvd.init()
# Pin GPU to be used to process local rank (one GPU per process)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')

In [9]:
def get_pipeline(model, optimizer=None):
    alphabet = asr.text.Alphabet(lang='en')
    features_extractor = asr.features.MFCC(
        features_num=26,
        is_standardization=False,
        winlen=0.032,
        winstep=0.02,
    )
    if not optimizer:
        optimizer = tf.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999)
    decoder = asr.decoder.GreedyDecoder()
    pipeline = asr.pipeline.CTCPipeline(
        alphabet, features_extractor, model, optimizer, decoder
    )
    return pipeline

In [10]:
def train_model(filename, dataset_idx, val_dataset_idx=None, epochs=25, tensorboard=False, restart_filename=None, use_mixed_precision=True):
    basename = os.path.basename(filename).split('.')[0]
    model_dir = os.path.join(os.path.dirname(filename), basename + '_train')
    os.makedirs(model_dir, exist_ok=True)
    
    model = keras.models.load_model(filename, custom_objects=TF_CUSTOM_OBJECTS)
    
    if restart_filename:
        model.load_weights(restart_filename)
    dataset = asr.dataset.Audio.from_csv(dataset_idx, batch_size=1, use_filesizes=True)
    if val_dataset_idx:
        val_dataset = asr.dataset.Audio.from_csv(val_dataset_idx, batch_size=1, use_filesizes=True)

    #opt_instance = tf.optimizers.Adam(0.0001 * hvd.size(), beta_1=0.9, beta_2=0.999)
    opt_instance = tfa.optimizers.NovoGrad(0.000, beta_1=0.95, beta_2=0.5, weight_decay=0.001)

    opt = hvd.DistributedOptimizer(opt_instance)
    pipeline = get_pipeline(model, opt)
    pipeline.compile_model(experimental_run_tf_function=False) # fix for the tf2.0 bug with horovod
    
    callbacks = [
        hvd.callbacks.BroadcastGlobalVariablesCallback(0),
        hvd.callbacks.MetricAverageCallback(),
    ]
    schedule=tf.keras.experimental.CosineDecayRestarts(
        0.000 * hvd.size(), 10, t_mul=2.0, m_mul=1.0, alpha=0.0,
    )
    callbacks.append(LearningRateScheduler(schedule))
    if hvd.rank() == 0:
        prefix = datetime.now().strftime("%Y%m%d-%H%M%S")
        monitor_metric_name = 'loss' # if not val_dataset_idx else 'val_loss'  # val_loss is wrong and broken
        callbacks.append(
            keras.callbacks.ModelCheckpoint(
                os.path.join(model_dir, prefix + '_best.ckpt'),
                monitor=monitor_metric_name, save_weights_only=True,
                save_best_only=True))
        if tensorboard:
            logdir = os.path.join(model_dir, 'tb', prefix)
            tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir, profile_batch = '8,10')
            callbacks.append(tensorboard_callback)

    time_start = time.time()

    hist = pipeline.fit(dataset, epochs=epochs, dev_dataset=val_dataset,
                        callbacks=callbacks, verbose=1 if hvd.rank() == 0 else 0,
                        steps_per_epoch=10, validation_steps=10)
    elapsed = time.time() - time_start
    
    if hvd.rank() == 0:
        print(f'Elapsed time: {elapsed}')
        #np.save(os.path.join(model_dir, prefix + '_hist.p'), np.array(hist))

In [11]:
train_model(
    filename='./models/tt_ds_uniform_initall_r14.h5', 
    dataset_idx='./train-clean-100-index.csv',
    val_dataset_idx='./dev-clean-index.csv',
    epochs=25,
    tensorboard=True,
    restart_filename=None,
)

Epoch 1/25
Instructions for updating:
Prefer tf.tensor_scatter_nd_add, which offers the same functionality with well-defined read-write semantics.
Instructions for updating:
Prefer tf.tensor_scatter_nd_update, which offers the same functionality with well-defined read-write semantics.
10/10 [==============================] - 411s 41s/step - loss: 709.0084 - val_loss: 688.8415 - lr: 0.0000e+00
Epoch 2/25
10/10 [==============================] - 372s 37s/step - loss: 694.7803 - val_loss: 708.6496 - lr: 0.0000e+00
Epoch 3/25
10/10 [==============================] - 326s 33s/step - loss: 672.1175 - val_loss: 708.3100 - lr: 0.0000e+00
Epoch 4/25
10/10 [==============================] - 358s 36s/step - loss: 708.9560 - val_loss: 708.6942 - lr: 0.0000e+00
Epoch 5/25
10/10 [==============================] - 367s 37s/step - loss: 709.2567 - val_loss: 708.7642 - lr: 0.0000e+00
Epoch 6/25
10/10 [==============================] - 381s 38s/step - loss: 709.3087 - val_loss: 708.4691 - lr: 0.0000e+00

In [ ]:
! ls models/ds_train

if __name__ == '__main__':
    parser = argparse.ArgumentParser(
        description='Train ctc asr model')
    parser.add_argument('--filename', type=str,
                        help='filename of the model')
    parser.add_argument('--dataset', type=str,
                        help='path to the dataset index',)
    parser.add_argument('--val_dataset', type=str,
                        help='path to the validation dataset index (optional)',
                        default=None)
    parser.add_argument('--epochs', type=int,
                       help='number of epochs to use for training',
                       default=25)
    parser.add_argument('--tensorboard', type=bool,
                       help='if tensorboard log will be written',
                       default=False)
    parser.add_argument('--restart_filename', type=str,
                       help='filename of the checkpoint to restart from',
                       default=None)
    args = parser.parse_args()
    
    train_model(args.filename, args.dataset,
                args.val_dataset, args.epochs,
                args.tensorboard, args.restart_filename)

training times: 25 epochs - 10 batch / epoch
 tt mixed - gpu - 9427s - 1 / batch
 

In [13]:
filename='./models/ds.h5'
dataset_idx='./train-clean-100-index.csv'
val_dataset_idx='./dev-clean-index.csv'
epochs=25
tensorboard=True
restart_filename=None
use_mixed_precision=True

In [14]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

In [15]:
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

Compute dtype: float16
Variable dtype: float32


In [16]:
model = keras.models.load_model(filename, custom_objects=TF_CUSTOM_OBJECTS)

In [17]:
model.layers

In [34]:
x = tf.zeros([1, 1, 2048])

In [35]:
y = model.layers[7](x)

In [36]:
y.dtype

tf.float32

In [38]:
from tensorflow.keras.mixed_precision import blah

ImportError: cannot import name 'blah' from 'tensorflow.keras.mixed_precision' (/trinity/home/r.schutski/asr_speedup/venv/lib/python3.7/site-packages/tensorflow/keras/mixed_precision/__init__.py)

> <ipython-input-38-3d12bbeb572b>(1)<module>()
----> 1 from tensorflow.keras.mixed_precision import blah



ipdb>  q
